Xception - Transfer Learning Model Implementation - Stefano Maxenti - Riccardo Mencucci - "team_durian"

In [ ]:
!pip install --upgrade pip
!pip install scipy sklearn pandas seaborn pillow visualkeras

In [2]:
import tensorflow as tf
import numpy as np
import os
import random
import time
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

2.6.0


In [3]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [4]:
DATA_ROOT = "DATASET_AUG"

In [5]:
COLOR_MODE = 'rgb'
BATCH_SIZE = 128

from tensorflow.keras.preprocessing.image import ImageDataGenerator

split_dir = DATA_ROOT
train_dir = os.path.join(split_dir,"train")
valid_dir = os.path.join(split_dir,"val")

# rescaling makes lookup fail?
# no, casting issues.
from tensorflow.keras.applications.xception import preprocess_input
train_data_generator = ImageDataGenerator(
                                            preprocessing_function=preprocess_input )

# keras onine augmentation drammatically increases training time
# dense training from < 1 min to > 5 min
valid_data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

train_gen = train_data_generator.flow_from_directory(
    directory = train_dir,
    # desired size for image, regardless of
    # image data. Images will be resized to fit
    # if necessary.
    # We use original size ==> no transfrom
    target_size = (256,256),
    # 3 color channel.
    # again, match data on disk.
    color_mode = COLOR_MODE,
    # default for classes is None.
    # That infers from name of folder classes the integer associated
    # with the class in lexicographical order.
    # This matches the order we used before.
    # We can change this by passing a list of strings to classes,
    # which should contain all (?) folder names. For now, the default one
    # will suffice.
    classes = None,
    # we need not specify this in model.fit, but we can do so here at data
    # generation.
    batch_size = BATCH_SIZE,
    shuffle = True
)

valid_gen = valid_data_generator.flow_from_directory(
    directory = valid_dir,
    target_size = (256,256),
    color_mode = COLOR_MODE,
    classes = None,
    batch_size = BATCH_SIZE,
    shuffle = False
)

Found 18541 images belonging to 14 classes.
Found 6187 images belonging to 14 classes.


In [ ]:
def get_next_batch(generator):
  batch = next(generator)

  image = batch[0]
  target = batch[1]

  print("(Input) image shape:", image.shape)
  print("Target shape:",target.shape)

  # Visualize only the first sample
  image = image[0]
  target = target[0]
  target_idx = np.argmax(target)
  print()
  print("Categorical label:", target)
  print("Label:", target_idx)
  #print("Class name:", labels[target_idx])
  fig = plt.figure(figsize=(6, 4))
  plt.imshow(np.uint8(image*255))

  return batch

_ = get_next_batch(valid_gen)

In [8]:
# Utility function to create folders and callbacks for training
from datetime import datetime

def create_folders_and_callbacks(model_name):

  exps_dir = os.path.join('XCEPTION')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%b%d_h%H_m%M_s%S')

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []

  # Model checkpoint
  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  ckpt_callback = tfk.callbacks.ModelCheckpoint(
      filepath=os.path.join(ckpt_dir, 'cp', "ckpt.hdf5"),
      save_weights_only=False,
      # if true, only save best epoch.
      # otherwise, save last epoch.
      # As we use early stopping, which already returns best model,
      # by setting false here he get both.

      save_best_only = True  
  )
  callbacks.append(ckpt_callback)

  # Visualize Learning on Tensorboard
  # ---------------------------------
  tb_dir = os.path.join(exp_dir, 'tb_logs')
  if not os.path.exists(tb_dir):
      os.makedirs(tb_dir)
      
  tb_callback = tfk.callbacks.TensorBoard(
      log_dir = tb_dir,
      # avoid profiling to increase runtime performance
      profile_batch = 0,
      histogram_freq = 1
  )
  callbacks.append(tb_callback)

  # Early Stopping
  # --------------
  es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
  callbacks.append(es_callback)

  return callbacks

In [15]:
input_shape = (256,256,3)
epochs_dense = 10
epochs_finetuning = 60

In [11]:
# transfer learning, import VGG and build model


supernet = tfk.applications.Xception(
    # remove classifier, only get fearure extractor
    include_top=False,
    weights="imagenet",
    input_shape=input_shape
)

supernet.trainable = False


inputs = tfk.Input(shape=input_shape)
x = supernet(inputs)
x = tfkl.GlobalAveragePooling2D(name='gpooling')(x)
#x = tfkl.Flatten(name='Flattening')(x)
x = tfkl.Dropout(0.3, seed=seed)(x)
x = tfkl.Dense(
    128, 
    activation='relu',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)
x = tfkl.Dropout(0.3, seed=seed)(x)
outputs = tfkl.Dense(
    14, 
    activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)

model = tfk.Model(inputs=inputs, outputs=outputs, name='model_transfer_learning_GAP')
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(1e-3), metrics='accuracy')



for i, layer in enumerate(model.get_layer('xception').layers):
   print(i, layer.name, layer.trainable)
model.summary()

2021-11-28 15:18:19.019924: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-28 15:18:19.029091: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-28 15:18:19.029695: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-28 15:18:19.031256: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-28 15:18:19.031846: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:1050] successful NUMA node read f

83697664/83683744 [==============================] - 4s 0us/step
0 input_1 False
1 block1_conv1 False
2 block1_conv1_bn False
3 block1_conv1_act False
4 block1_conv2 False
5 block1_conv2_bn False
6 block1_conv2_act False
7 block2_sepconv1 False
8 block2_sepconv1_bn False
9 block2_sepconv2_act False
10 block2_sepconv2 False
11 block2_sepconv2_bn False
12 conv2d False
13 block2_pool False
14 batch_normalization False
15 add False
16 block3_sepconv1_act False
17 block3_sepconv1 False
18 block3_sepconv1_bn False
19 block3_sepconv2_act False
20 block3_sepconv2 False
21 block3_sepconv2_bn False
22 conv2d_1 False
23 block3_pool False
24 batch_normalization_1 False
25 add_1 False
26 block4_sepconv1_act False
27 block4_sepconv1 False
28 block4_sepconv1_bn False
29 block4_sepconv2_act False
30 block4_sepconv2 False
31 block4_sepconv2_bn False
32 conv2d_2 False
33 block4_pool False
34 batch_normalization_2 False
35 add_2 False
36 block5_sepconv1_act False
37 block5_sepconv1 False
38 block5_sepcon

In [12]:
# Create folders and callbacks and fit
#model = tfk.models.load_model("C:/Users/Riccardo/Downloads/10526141_10534455_2R/SubmissionModel")
model.summary()
noaug_callbacks = create_folders_and_callbacks(model_name='transfer_gap')
print("starting at ", datetime.now())
# Train the model
history = model.fit(
    x = train_gen,
    epochs = epochs_dense,
    validation_data = valid_gen,
    callbacks = noaug_callbacks
).history

Model: "model_transfer_learning_GAP"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
xception (Functional)        (None, 8, 8, 2048)        20861480  
_________________________________________________________________
gpooling (GlobalAveragePooli (None, 2048)              0         
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               262272    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 14)

/usr/local/lib/python3.8/dist-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
2021-11-28 15:18:50.205879: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10


2021-11-28 15:18:53.307417: I tensorflow/stream_executor/cuda/cuda_dnn.cc:381] Loaded cuDNN version 8204


  1/145 [..............................] - ETA: 14:27 - loss: 2.6913 - accuracy: 0.1094

2021-11-28 15:18:56.247555: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


145/145 [==============================] - 104s 682ms/step - loss: 0.8070 - accuracy: 0.7497 - val_loss: 0.3142 - val_accuracy: 0.9150
Epoch 2/10
145/145 [==============================] - 58s 397ms/step - loss: 0.3674 - accuracy: 0.8866 - val_loss: 0.2303 - val_accuracy: 0.9350
Epoch 3/10
145/145 [==============================] - 63s 436ms/step - loss: 0.2932 - accuracy: 0.9059 - val_loss: 0.1857 - val_accuracy: 0.9429
Epoch 4/10
145/145 [==============================] - 57s 395ms/step - loss: 0.2508 - accuracy: 0.9192 - val_loss: 0.1677 - val_accuracy: 0.9501
Epoch 5/10
145/145 [==============================] - 58s 398ms/step - loss: 0.2200 - accuracy: 0.9298 - val_loss: 0.1596 - val_accuracy: 0.9517
Epoch 6/10
145/145 [==============================] - 57s 394ms/step - loss: 0.1998 - accuracy: 0.9337 - val_loss: 0.1389 - val_accuracy: 0.9557
Epoch 7/10
145/145 [==============================] - 64s 439ms/step - loss: 0.1825 - accuracy: 0.9404 - val_loss: 0.1326 - val_accuracy: 0.

In [13]:
model_file_name = "xception" + datetime.now().strftime('%b%d_h%H_m%M_s%S')
model.save(model_file_name)
print("model saved")

2021-11-28 15:30:56.882185: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: xceptionNov28_h15_m30_s51/assets


/usr/local/lib/python3.8/dist-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


model saved


In [14]:
# make last classifier layer trainable for fine tuning
model.get_layer('xception').trainable = True

for i, layer in enumerate(model.get_layer('xception').layers[:105]):
   layer.trainable = False

for i, layer in enumerate(model.get_layer('xception').layers):
   print(i, layer.name, layer.trainable)
model.summary()

# lower learning rate
from keras import backend as K

print("Learning rate before update:", model.optimizer.learning_rate.numpy())

K.set_value(model.optimizer.learning_rate, 1e-4)

print("Learning rate after update:", model.optimizer.learning_rate.numpy())

0 input_1 False
1 block1_conv1 False
2 block1_conv1_bn False
3 block1_conv1_act False
4 block1_conv2 False
5 block1_conv2_bn False
6 block1_conv2_act False
7 block2_sepconv1 False
8 block2_sepconv1_bn False
9 block2_sepconv2_act False
10 block2_sepconv2 False
11 block2_sepconv2_bn False
12 conv2d False
13 block2_pool False
14 batch_normalization False
15 add False
16 block3_sepconv1_act False
17 block3_sepconv1 False
18 block3_sepconv1_bn False
19 block3_sepconv2_act False
20 block3_sepconv2 False
21 block3_sepconv2_bn False
22 conv2d_1 False
23 block3_pool False
24 batch_normalization_1 False
25 add_1 False
26 block4_sepconv1_act False
27 block4_sepconv1 False
28 block4_sepconv1_bn False
29 block4_sepconv2_act False
30 block4_sepconv2 False
31 block4_sepconv2_bn False
32 conv2d_2 False
33 block4_pool False
34 batch_normalization_2 False
35 add_2 False
36 block5_sepconv1_act False
37 block5_sepconv1 False
38 block5_sepconv1_bn False
39 block5_sepconv2_act False
40 block5_sepconv2 False

In [16]:
history = model.fit(
    x = train_gen,
    initial_epoch = 10,
    epochs = epochs_finetuning,
    validation_data = valid_gen,
    callbacks = noaug_callbacks
).history

Epoch 11/60
145/145 [==============================] - 58s 399ms/step - loss: 0.1312 - accuracy: 0.9564 - val_loss: 0.1078 - val_accuracy: 0.9686
Epoch 12/60
145/145 [==============================] - 57s 392ms/step - loss: 0.1293 - accuracy: 0.9570 - val_loss: 0.1058 - val_accuracy: 0.9678
Epoch 13/60
145/145 [==============================] - 57s 394ms/step - loss: 0.1230 - accuracy: 0.9591 - val_loss: 0.1060 - val_accuracy: 0.9683
Epoch 14/60
145/145 [==============================] - 57s 393ms/step - loss: 0.1234 - accuracy: 0.9601 - val_loss: 0.1057 - val_accuracy: 0.9685
Epoch 15/60
145/145 [==============================] - 58s 399ms/step - loss: 0.1209 - accuracy: 0.9593 - val_loss: 0.1049 - val_accuracy: 0.9695
Epoch 16/60
145/145 [==============================] - 58s 400ms/step - loss: 0.1205 - accuracy: 0.9605 - val_loss: 0.1038 - val_accuracy: 0.9693
Epoch 17/60
145/145 [==============================] - 58s 398ms/step - loss: 0.1208 - accuracy: 0.9606 - val_loss: 0.1037 -

In [17]:
model_file_name = "xception_final_download" + datetime.now().strftime('%b%d_h%H_m%M_s%S')
model.save(model_file_name)
print("model saved")
os.system(f"zip -r best_model_final_exception.zip ./{model_file_name}")

INFO:tensorflow:Assets written to: xception_final_downloadNov28_h16_m23_s32/assets


/usr/local/lib/python3.8/dist-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


model saved
  adding: xception_final_downloadNov28_h16_m23_s32/ (stored 0%)
  adding: xception_final_downloadNov28_h16_m23_s32/assets/ (stored 0%)
  adding: xception_final_downloadNov28_h16_m23_s32/variables/ (stored 0%)
  adding: xception_final_downloadNov28_h16_m23_s32/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: xception_final_downloadNov28_h16_m23_s32/variables/variables.index (deflated 76%)
  adding: xception_final_downloadNov28_h16_m23_s32/saved_model.pb (deflated 93%)
  adding: xception_final_downloadNov28_h16_m23_s32/keras_metadata.pb (deflated 96%)


0

In [7]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

valid_gen.shuffle = False

Y_prediction = model.predict_generator(valid_gen, len(valid_gen))
# Convert predictions classes to one hot vectors 
Y_pred_classes = np.argmax(Y_prediction,axis = 1) 
# Convert validation observations to one hot vectors
Y_true = valid_gen.classes
# compute the confusion matrix
confusion_mtx = confusion_matrix(Y_true, Y_pred_classes)
class_report = classification_report(Y_true, Y_pred_classes, 
                                     target_names=valid_gen.class_indices.keys())  # target_names must be ordered depending on the class labels
print('Confusion Matrix:')
print(confusion_mtx)
print()
print('Classification Report:')
print(class_report)

model.evaluate(valid_gen, return_dict=True)

/usr/local/lib/python3.8/dist-packages/keras/engine/training.py:2035: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '
2021-11-28 16:58:47.327898: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-28 16:58:50.284425: I tensorflow/stream_executor/cuda/cuda_dnn.cc:381] Loaded cuDNN version 8204


Confusion Matrix:
[[ 344    5    2    0    0    3    2    2    1    0    3    3    0    7]
 [   1  232    0    0    0    1    0    1    1    2    2    0    1    1]
 [   5    0  253    0    0    2    0    2    0    0    5    0    0    4]
 [   0    0    0  427    0    0    0    0    0    0    0    0    0    0]
 [   2    0    0    0  475    0    0    0    1    0    0    0    5    7]
 [   0    0    0    0    0  559    1    1    0    0    0    0    0    1]
 [   3    0    0    0    0    0  363    1    0    0    0    1    0    2]
 [   3    2    1    0    0    1    1  301    1    0    2    0    0    5]
 [   2    1    0    0    1    0    1    1  289    0    4    0    0    5]
 [   0    0    0    0    1    0    0    0    0  189    0    0    0    1]
 [   3    0    4    0    0    0    0    2    0    0  518    0    0    2]
 [   0    0    0    0    0    0    0    0    0    0    0  265    1    3]
 [   2    1    0    0    2    0    0    0    3    2    0    0  278    6]
 [   7    2    4    0    5    0  

{'loss': 0.09110496938228607, 'accuracy': 0.9712299704551697}